## **실습 3.**

본 실습 자료는 **BDC103(00) 빅데이터와정보검색 강의 실습**을 위해 **고려대학교 자연어처리연구실 (NLP & AI Lab)**에서 제작했습니다.

☠️ 외부로의 무단 배포를 금지합니다. ☠️

```
version 1.0 (2021.05.16)
created by: 서재형, 임희석 (고려대학교 자연어처리 연구실)
email: wolhalang@gmail.com
```

```
HuggingFace의 모델과 학습 방법을 참조해서 제작했습니다.
https://huggingface.co/transformers/
```

※ 오늘은 GPU 환경에서의 실험을 위해서 

**런타임 > 런타임 유형 변경> 하드웨어 가속기 GPU**로 설정해주세요

### **구글 코랩과 구글 드라이브 연동하기** 😎

구글 코랩 환경에서는 로그인되어 있는 구글 계정과 연결되어 있는 '구글 드라이브'와의 연동을 지원합니다.  
연동하는 경우, 자신의 구글 드라이브를 로컬 환경처럼 활용이 가능하며, 원하는 파일이나 이미지를 불러올 수 있습니다.

In [ ]:
### Google Drive 패키지와 os 모듈 불러오기
from google.colab import drive
import os

In [ ]:
### 본인 구글 드라이브의 최초 경로를 설정하기

# 대부분의 구글 드라이브 최초 경로는 아래와 같습니다. 
# 예외 발생 시 본인의 구글 드라이브에 접속하여 content 폴더나 gdrive 폴더가 어떤 위치에 있으며, 
# 자신이 연결하려는 폴더까지의 경로가 어떻게 되는지 확인해야 합니다.  
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
### 자신의 현재 경로를 파악하기
os.getcwd() ## 아마도 /content로 나올 것.

'/content'

In [ ]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/information_retrieval')

### 🤥💦 모르는 것이 있을 경우에는?

프로그래밍의 첫 걸음은 **`검색`** 입니다.

특히 **`빅데이터와 정보 검색`** 수업을 듣고 있는 여러분들은 **반.드.시** 궁금하거나 모르는 내용에 대해서 **`검색`**하는 습관이 필요합니다.

> Google 검색 엔진 
                
    -> "Python 루프문"
                 
    -> "Python for문 에러"

    -> "What is SyntaxError: invalid syntax in python?"

등의 `에러문장`과 `키워드`를 적극 활용해주세요!😄


### **정보 검색 기반의 질의 응답 모델** 🧐

<img src = "https://github.com/facebookresearch/DrQA/raw/master/img/drqa.png" width = "600" >

> This is a PyTorch implementation of the DrQA system described in the ACL 2017 paper Reading Wikipedia to Answer Open-Domain Questions.

> ACL 2017. 페이스북에서 공개한 정보 검색 기반 질의 응답 시스템입니다. 

> 논문 링크: https://arxiv.org/abs/1704.00051

```
@inproceedings{chen2017reading,
  title={Reading {Wikipedia} to Answer Open-Domain Questions},
  author={Chen, Danqi and Fisch, Adam and Weston, Jason and Bordes, Antoine},
  booktitle={Association for Computational Linguistics (ACL)},
  year={2017}
}
```




### **정보 검색 기반의 질의 응답 예시**

**Example 1**
```
>>> process('What is question answering?')

Top Predictions:
+------+----------------------------------------------------------------------------------------------------------+--------------------+--------------+-----------+
| Rank |                                                  Answer                                                  |        Doc         | Answer Score | Doc Score |
+------+----------------------------------------------------------------------------------------------------------+--------------------+--------------+-----------+
|  1   | a computer science discipline within the fields of information retrieval and natural language processing | Question answering |    1917.8    |   327.89  |
+------+----------------------------------------------------------------------------------------------------------+--------------------+--------------+-----------+

Contexts:
[ Doc = Question answering ]
Question Answering (QA) is a computer science discipline within the fields of
information retrieval and natural language processing (NLP), which is
concerned with building systems that automatically answer questions posed by
humans in a natural language.
```
**Example 2**
```
>>> process('What is the answer to life, the universe, and everything?')

Top Predictions:
+------+--------+---------------------------------------------------+--------------+-----------+
| Rank | Answer |                        Doc                        | Answer Score | Doc Score |
+------+--------+---------------------------------------------------+--------------+-----------+
|  1   |   42   | Phrases from The Hitchhiker's Guide to the Galaxy |    47242     |   141.26  |
+------+--------+---------------------------------------------------+--------------+-----------+

Contexts:
[ Doc = Phrases from The Hitchhiker's Guide to the Galaxy ]
The number 42 and the phrase, "Life, the universe, and everything" have
attained cult status on the Internet. "Life, the universe, and everything" is
a common name for the off-topic section of an Internet forum and the phrase is
invoked in similar ways to mean "anything at all". Many chatbots, when asked
about the meaning of life, will answer "42". Several online calculators are
also programmed with the Question. Google Calculator will give the result to
"the answer to life the universe and everything" as 42, as will Wolfram's
Computational Knowledge Engine. Similarly, DuckDuckGo also gives the result of
"the answer to the ultimate question of life, the universe and everything" as
42. In the online community Second Life, there is a section on a sim called
43. "42nd Life." It is devoted to this concept in the book series, and several
attempts at recreating Milliways, the Restaurant at the End of the Universe, were made.
```

### **데이터셋**

#### **영어 기계 독해 기반의 질의 응답 데이터셋**

```
SQuAD 2.0: https://rajpurkar.github.io/SQuAD-explorer/
```

#### **한국어 기계 독해 기반의 질의 응답 데이터셋**

```
KorQuAD 1.0: https://korquad.github.io/KorQuad%201.0/

KorQuAD 2.0: https://korquad.github.io/
```

```
KorQuAD는 한국어 기반의 기계 독해를 위해 제작한 데이터셋이며, 데이터셋은 지문과 질의 그리고 선택지로 

이루어져 있습니다. 질의에 대한 답변은 위키피디아를 근거로 제작했으며, json 파일 형태로 훈련/검증/평가 데이터를 제공합니다.
```





In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json

--2021-05-17 10:03:54--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json.2’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  43.9MB/s    in 0.8s    

2021-05-17 10:03:56 (43.9 MB/s) - ‘KorQuAD_v1.0_train.json.2’ saved [38527475/38527475]



In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json

--2021-05-17 10:03:57--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json.1’

KorQuAD_v1.0_dev.js 100%[===================>]   3.70M  22.1MB/s    in 0.2s    

2021-05-17 10:03:57 (22.1 MB/s) - ‘KorQuAD_v1.0_dev.json.1’ saved [3881058/3881058]



In [ ]:
import json

with open('KorQuAD_v1.0_train.json', 'r', encoding='utf-8') as f:
    korquad_train = json.load(f)
    print(korquad_train.keys())
    #print(korquad_train.values()) # 로딩 시간이 조금 걸려요
    
    ## 0번째 타이틀에 대한 지문 집합
    print(korquad_train['data'][0])
    print(korquad_train['data'][0].keys()) # title, paragraphs (위키피디아를 기준으로 제작한 데이터임을 고려하기!)
    print(korquad_train['data'][0]['title']) # 제목은 하나 
    print(korquad_train['data'][0]['paragraphs']) 
    print(len(korquad_train['data'][0]['paragraphs'])) # 지문은 N개

    # 0번째 타이틀의 0번째 지문에 대한 {"질의응답": ~~ , "지문":   ~~}
    print(korquad_train['data'][0]['paragraphs'][0].keys()) # qas (question answering), context
    print(korquad_train['data'][0]['paragraphs'][0]['qas'])
    print(len(korquad_train['data'][0]['paragraphs'][0]['qas'])) # 질의 응답 쌍도 M개
    print(korquad_train['data'][0]['paragraphs'][0]['qas'][0]) # ex) 응답: , 질의: 
    print(korquad_train['data'][0]['paragraphs'][0]['qas'][0]['answers']) # answer_start의 경우 span을 의미
    print(korquad_train['data'][0]['paragraphs'][0]['qas'][0]['question'])

dict_keys(['version', 'data'])
{'paragraphs': [{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}], 'id': '6566495-0-0', 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'}, {'answers': [{'text': '1악장', 'answer_start': 421}], 'id': '6566495-0-1', 'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'}, {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}], 'id': '6566495-0-2', 'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'}, {'answers': [{'text': '파우스트', 'answer_start': 15}], 'id': '6566518-0-0', 'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'}, {'answers': [{'text': '합창교향곡', 'answer_start': 354}], 'id': '6566518-0-1', 'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'}, {'answers': [{'text': '1839', 'answer_start': 0}], 'id': '5917067-0-0', 'question': '바그너가 파우스트를 처음으로 읽은 년도는?'}, {'answers': [{'text': '파리', 'answer_start': 410}], 'id': '5917067-0-1', 'question': '바그너가 처음 교향곡 작곡을 한 장소는?'}, {'answers': [{'text': '드레스덴', 'answer_start': 534}], 'id': '5917067-0-2', 'question': '바그너의 1악장의 초연은 어디서 연주되

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json

--2021-05-17 10:03:58--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json.1’

train-v1.1.json.1   100%[===================>]  28.88M  41.0MB/s    in 0.7s    

2021-05-17 10:04:00 (41.0 MB/s) - ‘train-v1.1.json.1’ saved [30288272/30288272]



In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

--2021-05-17 10:04:00--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json.1’

dev-v1.1.json.1     100%[===================>]   4.63M  30.4MB/s    in 0.2s    

2021-05-17 10:04:01 (30.4 MB/s) - ‘dev-v1.1.json.1’ saved [4854279/4854279]



In [ ]:
with open('dev-v2.0.json', 'r', encoding='utf-8') as f:
    squad_dev = json.load(f)
    print(squad_dev.keys())
    #print(squad_dev.values()) # 로딩 시간이 조금 걸려요
    ## 0번째 타이틀에 대한 지문 집합
    print(squad_dev['data'][0])
    print(squad_dev['data'][0].keys()) # title, paragraphs (위키피디아를 기준으로 제작한 데이터임을 고려하기!)
    print(squad_dev['data'][0]['title']) # 제목은 하나 
    print(squad_dev['data'][0]['paragraphs']) 
    print(len(squad_dev['data'][0]['paragraphs'])) # 지문은 N개

    # 0번째 타이틀의 0번째 지문에 대한 {"질의응답": ~~ , "지문":   ~~}
    print(squad_dev['data'][0]['paragraphs'][0].keys()) # qas (question answering), context
    print(squad_dev['data'][0]['paragraphs'][0]['qas'])
    print(len(squad_dev['data'][0]['paragraphs'][0]['qas'])) # 질의 응답 쌍도 M개
    print(squad_dev['data'][0]['paragraphs'][0]['qas'][0]) # ex) 응답: , 질의: 
    print(squad_dev['data'][0]['paragraphs'][0]['qas'][0]['answers']) # answer_start의 경우 span을 의미
    print(squad_dev['data'][0]['paragraphs'][0]['qas'][0]['question'])

dict_keys(['version', 'data'])
{'title': 'Normans', 'paragraphs': [{'qas': [{'question': 'In what country is Normandy located?', 'id': '56ddde6b9a695914005b9628', 'answers': [{'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}], 'is_impossible': False}, {'question': 'When were the Normans in Normandy?', 'id': '56ddde6b9a695914005b9629', 'answers': [{'text': '10th and 11th centuries', 'answer_start': 94}, {'text': 'in the 10th and 11th centuries', 'answer_start': 87}, {'text': '10th and 11th centuries', 'answer_start': 94}, {'text': '10th and 11th centuries', 'answer_start': 94}], 'is_impossible': False}, {'question': 'From which countries did the Norse originate?', 'id': '56ddde6b9a695914005b962a', 'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256}, {'text': 'Denmark, Iceland and Norway', 'answer_start': 256}, {'text': 'Denmark, Iceland and Norway', 'answ

### **사전 훈련 언어모델 BERT**

BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
(https://arxiv.org/pdf/1810.04805.pdf)

<img src = "https://dos-tacos.github.io/images/lynn/181229/8.PNG" width = "600" >

- Transformer 기반의 양방향 오토 인코더 모델 (Auto Encoder)

(Vaswani et al., 2017 https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)


- 대용량의 텍스트 코퍼스를 통해서 사전 훈련된 모델로써 공개된 대표적인 언어 모델 중 하나.

(https://github.com/google-research/bert)

- BERT 사전 훈련의 특징

1) MASKED LM
- 기본 값 15% 무작위 확률로 [MASK] 입력 시퀀스에 대해서 wordpiece로 토큰화한 단어에 대해서 예측하도록 전체 문맥에 대해 양뱡향으로 정보를 습득하는 장점 극대화

ex) "나는 정보검색 [MASK]를 듣는다."

ex) "부산의 유명한 [MASK]에서 해수욕을 즐기고, 파도가 [MASK]지는 백사장을 거닐고 싶다."

2) Next Sentence Prediction
- 문장 단위의 입력 시퀀스를 넘어서, 문장과 문장 사이의 문맥을 파악할 수 있도록 하는 TASK를 포함

- 50%는 실제로 다음에 오는 문장, 50%는 잘못된 문장으로 옳은 문장이 다음에 오도록 예측하게 함.

- Special Tokens

[CLS]: 입력 시퀀스의 처음으로 분류 정보를 담기 위한 임베딩 토큰. 분류 문제를 해결하는 경우 최종 hidden state 값을 담고 있음.

[SEP]: 문장과 문장을 구분하여 하나의 시퀀스로 연결할 때 사용.

ex) [CLS] 나는 배가 고프다. [SEP] 밥을 먹으러 가야겠다. [SEP] 

<img src = "https://dos-tacos.github.io/images/lynn/181229/9.PNG" width = "600" >

```
이미지 참조
출처: https://dos-tacos.github.io/paper%20review/BERT/
```

### **사전 훈련된 BERT를 활용하여 질의 응답 문제 해결**

**BERT**

Google에서 2018년에 발표한 BERT 모델

공개된 사전 훈련 모델을 Pytorch 기반으로 사용하기 쉽게 만든 HuggingFace 참조

https://huggingface.co/transformers/

본 실습은 사전 훈련된 BERT를 바탕으로

미세 조정 훈련을 통해 SQuad 2.0의 질의 응답 문제를 해결

In [ ]:
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install utils_qa

ERROR: Could not find a version that satisfies the requirement utils_qa (from versions: none)
ERROR: No matching distribution found for utils_qa


In [ ]:
import argparse
import math
import os
import random
import datasets
import numpy as np
import torch
import collections
import json
import logging
import transformers

from tqdm.auto import tqdm
from datasets import load_dataset, load_metric
from torch.utils.data.dataloader import DataLoader
from typing import Optional, Tuple
from accelerate import Accelerator

# Huggingface transformers의 토크나이저, 설정파일, 최적화 함수 등 사용
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from utils_qa import postprocess_qa_predictions
from transformers.utils import check_min_version

In [ ]:
## 명령행 인터페이스, 사전 훈련된 모델을 로드하여 훈련하고 평가하는데까지 필요한 인자를 사용자 정의 인자로 설정하는 내용을 담고 있음.
## 사용법: python example.py --dataset_name squad.json .... 
## 주피터 환경을 위한 인자 설정법 
import easydict

def easydict_args():
    
    args = easydict.EasyDict({
        "dataset_name": 'squad',
        "dataset_config_name": None,
        "train_file": None,
        "preprocessing_num_workers": 4,
        "validation_file": None,
        "test_file": None,
        "max_seq_length": 384,
        "pad_to_max_length": None,
        "model_name_or_path": 'bert-base-cased',
        "config_name": None,
        "tokenizer_name": None,
        "use_slow_tokenizer": None,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 3e-5,
        "weight_decay": 0.0,
        "num_train_epochs": 3,
        "max_train_steps": None,
        "gradient_accumulation_steps": 1,
        "lr_scheduler_type": 'linear',
        "num_warmup_steps":0,
        "output_dir": 'result/',
        "seed": 42,
        "doc_stride": 128,
        "n_best_size": 20,
        "null_score_diff_threshold": 0.0,
        "version_2_with_negative": False,
        "max_answer_length": 30,
        "max_train_samples": None,
        "max_eval_samples": None,
        "overwrite_cache": False,
        "max_predict_samples": None,
        "do_predict": False
    })

    ## 무결성 체크
    ## 인자로 전달한 데이터셋 명이 잘못되었거나 확장자 오류가 있을 경우 raise 이하의 메세지를 출력
    if (
        args.dataset_name is None
        and args.train_file is None
        and args.validation_file is None
        and args.test_file is None
    ):
        raise ValueError("Need either a dataset name or a training/validation/test file.")
    else:
        if args.train_file is not None:
            extension = args.train_file.split(".")[-1]
            assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
        if args.validation_file is not None:
            extension = args.validation_file.split(".")[-1]
            assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if args.test_file is not None:
            extension = args.test_file.split(".")[-1]
            assert extension in ["csv", "json"], "`test_file` should be a csv or a json file."

    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
    
    ## 위에서 인자로 전달받은 값을 args 변수에 담아 반환
    return args

In [ ]:
logger = logging.getLogger(__name__)
def main():
    args = easydict_args()

    # Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
    ## 가속화 모드를 지원하며, 해당 모드로 병목이 발생하는 메서드 부분을 대체해서 사용가능.
    accelerator = Accelerator()
    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state)

    # Setup logging, we only want one process per machine to log things on the screen.
    # accelerator.is_local_main_process is only True for one process per machine.
    logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # If passed along, set the training seed now.
    if args.seed is not None:
        set_seed(args.seed)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
    # 'text' is found. You can easily tweak this behavior (see below).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    ## Huggingface의 datasets에 존재하는 데이터셋 명을 사용하는 경우에는 저장된 형식에 맞추어서 로드 (위의 링크에서 리스트 참조)
    if args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
    ## 사용자 정의의 데이터셋을 사용하는 경우에는 아래의 조건문 실행. 
    else:
        data_files = {}
        if args.train_file is not None:
            data_files["train"] = args.train_file
        if args.validation_file is not None:
            data_files["validation"] = args.validation_file
        if args.test_file is not None:
            data_files["test"] = args.test_file
        extension = args.train_file.split(".")[-1]
        raw_datasets = load_dataset(extension, data_files=data_files, field="data")
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Load pretrained model and tokenizer
    #
    # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    ## 지정한 모델의 설정에 맞추어서 환경 구성 요소 로드
    ## ex) BERT --> BERTConfig...
    if args.config_name:
        config = AutoConfig.from_pretrained(args.config_name)
    elif args.model_name_or_path:
        config = AutoConfig.from_pretrained(args.model_name_or_path)
    else:
        config = CONFIG_MAPPING[args.model_type]()
        logger.warning("You are instantiating a new config instance from scratch.")
    
    ## 지정한 모델의 설정에 맞추어서 토크나이저 로드
    ## ex) BERT --> BERTTokenizer ... 30,000개의 vocab와 word piece tokenzier를 곁들인...
    if args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=True)
    elif args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=True)
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported by this script."
            "You can do it from another script, save it, and load it from here, using --tokenizer_name."
        )

    ## 지정한 모델 로드하기.
    ## 여기서는 사전 훈련된 모델 상단에 질의응답 TASK를 위한 레이어를 추가하고 미세조정훈련을 하기위해 ForQuestionAnswering을 불러온다.
    if args.model_name_or_path:
        model = AutoModelForQuestionAnswering.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForQuestionAnswering.from_config(config)

    # Preprocessing the datasets.
    # Preprocessing is slighlty different for training and evaluation.
    
    ## 전처리하기 
    column_names = raw_datasets["train"].column_names
    print(column_names)
    #print(column_names)
    question_column_name = "question" if "question" in column_names else column_names[0]
    context_column_name = "context" if "context" in column_names else column_names[1]
    answer_column_name = "answers" if "answers" in column_names else column_names[2]

    # Padding side determines if we do (question|context) or (context|question).
    pad_on_right = tokenizer.padding_side == "right"

    ## 인자로 전달한 최대 입력 시퀀스 길이가 모델의 최대 입력 임베딩 길이보다 길 경우 에러 반환
    if args.max_seq_length > tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )

    max_seq_length = min(args.max_seq_length, tokenizer.model_max_length)

    ##################

    # Training preprocessing
    ## 훈련 데이터셋에 대한 전처리 작업
    ## 패드의 방향을 오른쪽으로 하는 것이 일반적이지만, 최근에는 다른 방향에도 추가할 수 있도록 기능 추가.
    
    def prepare_train_features(examples):
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=args.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length" if args.pad_to_max_length else False,
        )
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        ## span의 여부 체크
        offset_mapping = tokenized_examples.pop("offset_mapping")

        #print("offset_mapping", offset_mapping)

        # Let's label those examples!
        ## span 가능하도록 본문에서 정답이 존재하는 시작점, 끝점을 매핑하여 저장
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            # If no answers are given, set the cls_index as answer.
            ## answer span이 없을 경우에는 [CLS]로 대체
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        return tokenized_examples
    
    ######################
    ## 사용자 정의 훈련 데이터를 입력 인자로 사용하는 경우, --do_train을 입력 인자로 반드시 사용해야함.
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = raw_datasets["train"]
    ## 디버깅용
    if args.max_train_samples is not None:
        # We will select sample from whole data if agument is specified
        train_dataset = train_dataset.select(range(args.max_train_samples))
    
    ## 로드한 사용자 정의 훈련 데이터셋을 입력 데이터로 인코딩
    # Create train feature from dataset
    train_dataset = train_dataset.map(
        prepare_train_features,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
    )
    if args.max_train_samples is not None:
        # Number of samples might increase during Feature Creation, We select only specified max samples
        train_dataset = train_dataset.select(range(args.max_train_samples))

    ######################
    ## 위의 훈련 데이터와 동일
    # Validation preprocessing
    def prepare_validation_features(examples):
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=args.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length" if args.pad_to_max_length else False,
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples

    #################
    ## 사용자 정의 검증 데이터를 입력 인자로 사용하는 경우, --do_eval을 입력 인자로 반드시 사용해야함.
    if "validation" not in raw_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_examples = raw_datasets["validation"]
    if args.max_eval_samples is not None:
        # We will select sample from whole data
        eval_examples = eval_examples.select(range(args.max_eval_samples))
    # Validation Feature Creation
    eval_dataset = eval_examples.map(
        prepare_validation_features,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
    )

    if args.max_eval_samples is not None:
        # During Feature creation dataset samples might increase, we will select required samples again
        eval_dataset = eval_dataset.select(range(args.max_eval_samples))

    #################
    ## 실제 정답을 반환하고 싶을 경우
    ##  --do_predict을 입력 인자로 전달하는 경우, 사용자 정의 평가 데이터를 입력 인자로 전달해야함. 
    if args.do_predict:
        if "test" not in raw_datasets:
            raise ValueError("--do_predict requires a test dataset")
        predict_examples = raw_datasets["test"]
        if args.max_predict_samples is not None:
            # We will select sample from whole data
            predict_examples = predict_examples.select(range(args.max_predict_samples))
        # Predict Feature Creation
        predict_dataset = predict_examples.map(
            prepare_validation_features,
            batched=True,
            num_proc=args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not args.overwrite_cache,
        )
        if args.max_predict_samples is not None:
            # During Feature creation dataset samples might increase, we will select required samples again
            predict_dataset = predict_dataset.select(range(args.max_predict_samples))

    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    #################
    ## 입력 인코딩을 완료한 데이터셋에 대해서 batch_size 만큼 로드하기 (pytorch 프레임워크의 장점)
    # DataLoaders creation:
    if args.pad_to_max_length:
        # If padding was already done ot max length, we use the default data collator that will just convert everything
        # to tensors.
        data_collator = default_data_collator
    else:
        # Otherwise, `DataCollatorWithPadding` will apply dynamic padding for us (by padding to the maximum length of
        # the samples passed). When using mixed precision, we add `pad_to_multiple_of=8` to pad all tensors to multiple
        # of 8s, which will enable the use of Tensor Cores on NVIDIA hardware with compute capability >= 7.5 (Volta).
        data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=(8 if accelerator.use_fp16 else None))

    ## 훈련 데이터셋에 대한 로더 (지정한 배치 사이즈 만큼만 로드해서 훈련하기)
    train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size
    )

    ## 검증 데이터셋에 대한 로더
    eval_dataset_for_model = eval_dataset.remove_columns(["example_id", "offset_mapping"])
    eval_dataloader = DataLoader(
        eval_dataset_for_model, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
    )

    ## 실제 평가를 위해서 정답의 근거를 삭제한 형태로 inference
    if args.do_predict:
        predict_dataset_for_model = predict_dataset.remove_columns(["example_id", "offset_mapping"])
        predict_dataloader = DataLoader(
            predict_dataset_for_model, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
        )

    ## huggingface에서 제공하는 간편하게 평가 결과를 반환하도록 하는 함수 
    ## 깃헙에서 다운로드 받은 utils_qa.py를 참조
    # Post-processing:
    def post_processing_function(examples, features, predictions, stage="eval"):
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions = postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            version_2_with_negative=args.version_2_with_negative,
            n_best_size=args.n_best_size,
            max_answer_length=args.max_answer_length,
            null_score_diff_threshold=args.null_score_diff_threshold,
            output_dir=args.output_dir,
            prefix=stage,
        )
        # Format the result to the format the metric expects.
        if args.version_2_with_negative:
            formatted_predictions = [
                {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
        else:
            formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

        references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)

    ## F1 score, Exact match
    metric = load_metric("squad_v2" if args.version_2_with_negative else "squad")

    #################
    # Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
    def create_and_fill_np_array(start_or_end_logits, dataset, max_len):
        """
        Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
        Args:
            start_or_end_logits(:obj:`tensor`):
                This is the output predictions of the model. We can only enter either start or end logits.
            eval_dataset: Evaluation dataset
            max_len(:obj:`int`):
                The maximum length of the output tensor. ( See the model.eval() part for more details )
        """

        step = 0
        # create a numpy array and fill it with -100.
        ## pad에 해당하는 logit 값을 -100으로 지정
        ## 효율적인 배치 계산을 위한 방법으로 지정한 max sequence length에 메모리 효율을 올리는 방법
        logits_concat = np.full((len(dataset), max_len), -100, dtype=np.float64)
        # Now since we have create an array now we will populate it with the outputs gathered using accelerator.gather
        for i, output_logit in enumerate(start_or_end_logits):  # populate columns
            # We have to fill it such that we have to take the whole tensor and replace it on the newly created array
            # And after every iteration we have to change the step

            batch_size = output_logit.shape[0]
            cols = output_logit.shape[1]

            if step + batch_size < len(dataset):
                logits_concat[step : step + batch_size, :cols] = output_logit
            else:
                logits_concat[step:, :cols] = output_logit[: len(dataset) - step]

            step += batch_size

        return logits_concat

    # Optimizer
    # Split weights in two groups, one with weight decay and the other not.
    ## 최적화 과정에서 L2 Loss를 통해서 지나친 과적합을 방지하기 위한 부분. 
    ## L2 Regularization, 기존 손실 함수 값에 모든 학습 가능한 파라미터를 제곱하여 합하고 
    ## 기정한 람다 값만큼 영향도를 설정하는데 이때 람다는 정규화의 세기를 결정하는 하이퍼 파라미터로 
    ## 여기서는 weight_decay에 해당
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    ## 아담 최적화 함수 사용
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

    # Prepare everything with our `accelerator`.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

    # Note -> the training dataloader needs to be prepared before we grab his length below (cause its length will be
    # shorter in multiprocess)

    # Scheduler and math around the number of training steps.
    ## 가상의 미니배치사용. 최종 업데이트는 각 가상 미니 배치의 gradient 값을 모두 고려한 global gradient
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
    if args.max_train_steps is None:
        args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
    else:
        args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

    ## scheduler를 통해서 learning rate의 속도 조절.
    ## 처음에는 크게, 점점 약해지도록하며 local minimum으로 추정되는 구간에서는 크게 지정한 횟수 만큼 적용 가능.
    lr_scheduler = get_scheduler(
        name=args.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=args.max_train_steps,
    )

    # Train!
    total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Num Epochs = {args.num_train_epochs}")
    logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {args.max_train_steps}")

    # Only show the progress bar once on each machine.
    progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
    completed_steps = 0

    for epoch in range(args.num_train_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            loss = loss / args.gradient_accumulation_steps
            accelerator.backward(loss)
            if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                completed_steps += 1

            if completed_steps >= args.max_train_steps:
                break

    # Validation
    all_start_logits = []
    all_end_logits = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
            start_logits = outputs.start_logits # 모든 토큰 중에서 가장 probability가 높은 시작점
            end_logits = outputs.end_logits# 모든 토큰 중에서 가장 probability가 높은 끝점
            
            if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                end_logits = accelerator.pad_across_processes(end_logits, dim=1, pad_index=-100)

            all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
            all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

    max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor

    # concatenate the numpy array
    start_logits_concat = create_and_fill_np_array(all_start_logits, eval_dataset, max_len)
    end_logits_concat = create_and_fill_np_array(all_end_logits, eval_dataset, max_len)

    # delete the list of numpy arrays
    del all_start_logits
    del all_end_logits

    outputs_numpy = (start_logits_concat, end_logits_concat)
    prediction = post_processing_function(eval_examples, eval_dataset, outputs_numpy)
    eval_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
    logger.info(f"Evaluation metrics: {eval_metric}")

    # Prediction
    if args.do_predict:
        all_start_logits = []
        all_end_logits = []
        for step, batch in enumerate(predict_dataloader):
            with torch.no_grad():
                outputs = model(**batch)
                start_logits = outputs.start_logits
                end_logits = outputs.end_logits

                if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                    start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                    end_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)

                all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
                all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

        max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor
        # concatenate the numpy array
        start_logits_concat = create_and_fill_np_array(all_start_logits, predict_dataset, max_len)
        end_logits_concat = create_and_fill_np_array(all_end_logits, predict_dataset, max_len)

        # delete the list of numpy arrays
        del all_start_logits
        del all_end_logits

        outputs_numpy = (start_logits_concat, end_logits_concat)
        prediction = post_processing_function(predict_examples, predict_dataset, outputs_numpy)
        predict_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
        logger.info(f"Predict metrics: {predict_metric}")

    if args.output_dir is not None:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(args.output_dir, save_function=accelerator.save)



In [ ]:
import easydict
if __name__ == '__main__':
    main()

05/17/2021 10:07:21 - INFO - __main__ -   Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Use FP16 precision: False

05/17/2021 10:07:22 - WARNING - datasets.builder -   Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4fffa6cf76083860f85fa83486ec3028e7e32c342c218ff2a620fc6b2868483a)
loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "b

['answers', 'context', 'id', 'question', 'title']
 

05/17/2021 10:08:33 - INFO - __main__ -   Sample 83810 of the training set: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'end_positions': 74, 'input_ids': [101, 2627, 3004, 1111, 1103, 23816, 9257, 136, 102, 1752, 117, 1103, 23816, 9257, 1449, 24771, 1103, 1322, 1104, 2268, 17143, 14678, 1107, 13838, 1764, 4544, 119, 1966, 1103, 18985, 1116, 1105, 2565, 6433, 9099, 18942, 1113, 1112, 170, 13734, 1113, 3979, 117, 7544, 15396, 1103, 23816, 9257, 9389, 1245, 1103, 13838, 1433, 112, 188, 1260, 14277, 1148, 118, 1413, 2830, 119, 2307, 117, 1103, 23816, 9257, 9389, 1127, 14395, 1194, 5586, 1884, 12929, 1116, 1105, 1127, 

KeyboardInterrupt: ignored

## 👍 실습 종료!! 정말 고생 많으셨습니다.

## 🤥💦 모르는 것이 있을 경우에는? (Remind)

프로그래밍의 첫 걸음은 `검색` 입니다.

특히 `빅데이터와 정보 검색` 수업을 듣고 있는 여러분들은 반.드.시 궁금하거나 모르는 내용에 대해서 `검색`하는 습관이 필요합니다.

> Google 검색 엔진 
                
    -> "Python 루프문"
                 
    -> "Python for문 에러"

    -> "What is SyntaxError: invalid syntax in python?"

등의 `에러문장`과 `키워드`를 적극 활용해주세요!😄

